In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

In [3]:
%load_ext sql

In [5]:
%sql ibm_db_sa://blg11630:jsnv%2Bw47652s064m@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

In [6]:
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
BLG11630,CENSUS_DATA,2021-05-31 20:48:27.134126
BLG11630,CHICAGO_PUBLIC_SCHOOLS,2021-05-31 20:50:10.198124
BLG11630,CHICAGO_CRIME_DATA,2021-05-31 20:51:23.002158
BLG11630,US_FOMC_RATES,2021-06-02 14:51:56.612101
BLG11630,US_ISM_MANUFACTURING_PMI,2021-06-02 14:53:46.824188
BLG11630,US_CPI,2021-06-02 14:49:53.314324
BLG11630,US_ISM_SERVICES_PMI,2021-06-02 14:54:59.504757
BLG11630,US_NFP_RATES,2021-06-02 14:55:56.584638
BLG11630,US_TRADE_BALANCE,2021-06-02 14:57:34.241315
BLG11630,US_RETAIL_SALES,2021-06-02 14:58:36.464754


In [36]:
features = ['US_TRADE_BALANCE', 'US_NFP_RATES']
features_dict = {}
x = %sql select * from US_TRADE_BALANCE
y = %sql select * from US_NFP_RATES
features_dict['US_TRADE_BALANCE'] = pd.DataFrame(x, columns = ['Date','US_TRADE_BALANCE'])
features_dict['US_NFP_RATES'] = pd.DataFrame(y, columns = ['Date','US_NFP_RATES'])

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.
 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


In [37]:
features_dict['US_TRADE_BALANCE'].head()

,Date,US_TRADE_BALANCE
0,2021-05-04,-74.4
1,2021-04-07,-71.1
2,2021-03-05,-68.2
3,2021-02-05,-66.6
4,2021-01-07,-68.1


In [38]:
for value in features_dict.values():
    value['Year'] = pd.to_datetime(value['Date']).dt.year.astype('str') 
    value['Month'] = pd.to_datetime(value['Date']).dt.month.astype('str')
    value['Date_time'] = value['Year']+'-'+value['Month']
    value.drop(['Year','Month'], 1, inplace=True)
    value.dropna(axis=0, inplace=True)

In [39]:
features_dict['US_TRADE_BALANCE'].head()

,Date,US_TRADE_BALANCE,Date_time
0,2021-05-04,-74.4,2021.0-5.0
1,2021-04-07,-71.1,2021.0-4.0
2,2021-03-05,-68.2,2021.0-3.0
3,2021-02-05,-66.6,2021.0-2.0
4,2021-01-07,-68.1,2021.0-1.0


In [40]:
features_dict['US_NFP_RATES'].head()

,Date,US_NFP_RATES,Date_time
0,2021-05-07,0.061,2021.0-5.0
1,2021-04-02,0.060,2021.0-4.0
2,2021-03-05,0.062,2021.0-3.0
3,2021-02-05,0.063,2021.0-2.0
4,2021-01-08,0.067,2021.0-1.0


In [41]:
features_dict1 = features_dict.copy()
#features_dict1

In [42]:
for value in features_dict1.values():
    value.drop('Date', 1, inplace=True)

In [56]:
dataset = pd.concat(features_dict1.values(), axis=1)
dataset.head()

,US_TRADE_BALANCE,Date_time,US_NFP_RATES,Date_time
0,-74.4,2021.0-5.0,0.061,2021.0-5.0
1,-71.1,2021.0-4.0,0.060,2021.0-4.0
2,-68.2,2021.0-3.0,0.062,2021.0-3.0
3,-66.6,2021.0-2.0,0.063,2021.0-2.0
4,-68.1,2021.0-1.0,0.067,2021.0-1.0


In [57]:
dataset = dataset.loc[:,~dataset.columns.duplicated()]
dataset.head()

,US_TRADE_BALANCE,Date_time,US_NFP_RATES
0,-74.4,2021.0-5.0,0.061
1,-71.1,2021.0-4.0,0.060
2,-68.2,2021.0-3.0,0.062
3,-66.6,2021.0-2.0,0.063
4,-68.1,2021.0-1.0,0.067


In [48]:
label = %sql select * from US_FOMC_RATES
label = pd.DataFrame(label, columns = ['Date','US_FOMC_RATES'])
label['Year'] = pd.to_datetime(label['Date']).dt.year.astype('str') 
label['Month'] = pd.to_datetime(label['Date']).dt.month.astype('str')
label['Date_time'] = label['Year']+'-'+label['Month']
label.drop(['Year','Month'], 1, inplace=True)
label.dropna(axis=0, inplace=True)

 * ibm_db_sa://blg11630:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


In [50]:
label1 = label.copy()
label1.drop('Date', 1, inplace=True)
label1.head()

,US_FOMC_RATES,Date_time
0,0.0025,2021.0-4.0
1,0.0025,2021.0-3.0
2,0.0025,2021.0-1.0
3,0.0025,2020.0-12.0
4,0.0025,2020.0-11.0


In [58]:
dataset = dataset.merge(label1, on='Date_time', how='inner')
dataset.drop('Date_time', 1, inplace=True)        

In [59]:
dataset

,US_TRADE_BALANCE,US_NFP_RATES,US_FOMC_RATES
0,-71.1,0.060,0.0025
1,-68.2,0.062,0.0025
2,-68.1,0.067,0.0025
3,-63.1,0.067,0.0025
4,-63.9,0.069,0.0025
...,...,...,...
86,-42.8,0.096,0.0025
87,-49.9,0.095,0.0025
88,-40.3,0.097,0.0025
89,-39.7,0.097,0.0025
